# H1B Labor Condition Applications (Form ETA-9035) 

In [2]:
# H1B Labor Condition Applications (Form ETA-9035)

# http://econdataus.com/h1bdata.htm

#source: US Department of Labor
#https://www.dol.gov/agencies/eta/foreign-labor/performance

#source: US Bureau of Labor Statistics 
#https://www.bls.gov/oes/tables.htm

# metadata: lca_cols
# https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Record_Layout_FY2022_Q3.pdf

# DOL Guidance on Determining OES Wage Levels
# https://www.aila.org/infonet/dol-guidance-on-determining-oes-wage-levels

# 
# https://www.dol.gov/sites/doPrevailing Wage Determination Policy Guidancelgov/files/ETA/oflc/pdfs/Policy_Nonag_Progs.pdf

# git pull https://github.com/JohnBroberg/H1B_LCA.git

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## global variables

In [3]:

#columns to be read from LCA Excel file
lca_cols = ['CASE_NUMBER','CASE_STATUS', 'RECEIVED_DATE', 'DECISION_DATE', 'ORIGINAL_CERT_DATE'
           , 'VISA_CLASS', 'SOC_CODE', 'SOC_TITLE', 'FULL_TIME_POSITION'
           , 'TOTAL_WORKER_POSITIONS', 'EMPLOYER_NAME', 'NAICS_CODE', 'WORKSITE_WORKERS'
           , 'SECONDARY_ENTITY', 'SECONDARY_ENTITY_BUSINESS_NAME', 'WORKSITE_STATE'
           , 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY'
           , 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL']
#           , 'PW_OES_YEAR', 'PW_OTHER_SOURCE', 'PW_OTHER_YEAR', 'PW_SURVEY_PUBLISHER', 'PW_SURVEY_NAME']



# Fiscal Year 2023

In [4]:
# Download LCA_Disclosure_Data_FY2022_Q4.xlsx

df23 = pd.read_excel("https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2023_Q2.xlsx"
                  , usecols = lca_cols)
#                  , index_col = None)

df23 = df23[(df23['VISA_CLASS']=='H-1B') & \
        (df23['FULL_TIME_POSITION']=='Y')]

df23

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,...,WORKSITE_WORKERS,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_STATE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL
0,I-200-23090-892448,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,1,No,NaN,CA,113485.0,131000.0,Year,111322.0,Year,II
1,I-200-23090-892621,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,1,Yes,"FedEx Corporate Services, Inc.",NY,109283.0,NaN,Year,109283.0,Year,II
2,I-200-23090-894024,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,1,No,NaN,NC,115000.0,125000.0,Year,101109.0,Year,II
3,I-200-23090-894211,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2141.00,Mechanical Engineers,Y,1,...,1,No,NaN,CA,75000.0,NaN,Year,70221.0,Year,I
4,I-200-23090-892630,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2112.00,Industrial Engineers,Y,1,...,1,No,NaN,MI,100000.0,NaN,Year,90667.0,Year,III
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231539,I-200-22111-091232,Certified - Withdrawn,2022-04-20,2022-10-01,2022-04-27,H-1B,29-1069.03,Hospitalists,Y,1,...,1,No,NaN,OR,253050.0,NaN,Year,208000.0,Year,NaN
231540,I-200-22104-068433,Certified - Withdrawn,2022-04-13,2022-10-01,2022-04-20,H-1B,15-1132.00,"Software Developers, Applications",Y,1,...,1,No,NaN,TX,110000.0,180000.0,Year,109138.0,Year,III
231541,I-200-21208-486017,Certified - Withdrawn,2021-07-27,2022-10-01,2021-08-03,H-1B,15-2041.01,Biostatisticians,Y,1,...,1,No,NaN,NH,75733.0,85733.0,Year,75733.0,Year,I
231542,I-200-21043-075850,Certified - Withdrawn,2021-02-12,2022-10-01,2021-02-22,H-1B,13-2099.01,Financial Quantitative Analysts,Y,1,...,1,No,NaN,TX,125000.0,175000.0,Year,67371.0,Year,II


In [5]:
min(df23['DECISION_DATE'])

Timestamp('2022-10-01 00:00:00')

In [6]:
max(df23['DECISION_DATE'])

Timestamp('2023-03-31 00:00:00')

In [7]:
# no duplicate rows
len(df23)-len(df23.drop_duplicates())

0

In [8]:
# no duplicate Case Numbers
len(df23['CASE_NUMBER'])-len(df23['CASE_NUMBER'].drop_duplicates())

0

In [9]:
# remove duplcate cases, keeping row with latest Decision Date

#df23 = df23.sort_values(['CASE_NUMBER', 'DECISION_DATE'], ascending = False) \
#    .drop_duplicates(subset = 'CASE_NUMBER', keep = 'first', ignore_index = True)

In [10]:
#len(df23['CASE_NUMBER'])-len(df23['CASE_NUMBER'].drop_duplicates())

In [12]:
#dup = df23[df23.duplicated(['CASE_NUMBER'], keep = False)]\
#     .sort_values(['CASE_NUMBER', 'DECISION_DATE'], ascending = False)

#dup

In [13]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221966 entries, 0 to 231543
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   CASE_NUMBER                     221966 non-null  object        
 1   CASE_STATUS                     221966 non-null  object        
 2   RECEIVED_DATE                   221966 non-null  datetime64[ns]
 3   DECISION_DATE                   221966 non-null  datetime64[ns]
 4   ORIGINAL_CERT_DATE              15520 non-null   datetime64[ns]
 5   VISA_CLASS                      221966 non-null  object        
 6   SOC_CODE                        221966 non-null  object        
 7   SOC_TITLE                       221966 non-null  object        
 8   FULL_TIME_POSITION              221966 non-null  object        
 9   TOTAL_WORKER_POSITIONS          221966 non-null  int64         
 10  EMPLOYER_NAME                   221966 non-null  object 

In [14]:
# Add WORKSITE_EMPLOYER column

df23['WORKSITE_EMPLOYER'] = df23['EMPLOYER_NAME']
df23.loc[df23['SECONDARY_ENTITY'] == 'Yes', 'WORKSITE_EMPLOYER'] = df23['SECONDARY_ENTITY_BUSINESS_NAME']

df23[['EMPLOYER_NAME','SECONDARY_ENTITY_BUSINESS_NAME','WORKSITE_EMPLOYER']]

,EMPLOYER_NAME,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_EMPLOYER
0,Thermo Fisher Scientific Inc.,NaN,Thermo Fisher Scientific Inc.
1,ATOS SYNTEL INC,"FedEx Corporate Services, Inc.","FedEx Corporate Services, Inc."
2,"OPTCAPITAL, LLC",NaN,"OPTCAPITAL, LLC"
3,"EXCELLENCE OPTO, INC.",NaN,"EXCELLENCE OPTO, INC."
4,Yanfeng International Automotive Technology US...,NaN,Yanfeng International Automotive Technology US...
...,...,...,...
231539,Asante Physician Partners,NaN,Asante Physician Partners
231540,"Cloud Technologies, Inc.",NaN,"Cloud Technologies, Inc."
231541,DARTMOUTH-HITCHCOCK CLINIC,NaN,DARTMOUTH-HITCHCOCK CLINIC
231542,AI MX2 (US) LLC,NaN,AI MX2 (US) LLC


In [15]:
df23['WORKSITE_EMPLOYER'] = df23['WORKSITE_EMPLOYER'].str.upper()
df23['WORKSITE_EMPLOYER']

0                            THERMO FISHER SCIENTIFIC INC. 
1                            FEDEX CORPORATE SERVICES, INC.
2                                           OPTCAPITAL, LLC
3                                     EXCELLENCE OPTO, INC.
4         YANFENG INTERNATIONAL AUTOMOTIVE TECHNOLOGY US...
                                ...                        
231539                            ASANTE PHYSICIAN PARTNERS
231540                             CLOUD TECHNOLOGIES, INC.
231541                           DARTMOUTH-HITCHCOCK CLINIC
231542                                     AI MX2 (US) LLC 
231543                              LENDINGCLUB CORPORATION
Name: WORKSITE_EMPLOYER, Length: 221966, dtype: object

In [16]:
# Facebook changes company name to Meta
# https://www.cnbc.com/2021/10/28/facebook-changes-company-name-to-meta.html

dict_emp = {'FACEBOOK INC':'META PLATFORMS INC'
                , 'AMAZON CORPORATE LLC':'AMAZON COM SERVICES LLC'
                , 'AMAZON WEB SERVICES INC':'AMAZON COM SERVICES LLC'
                , 'AMAZON.COM SERVICES LLC':'AMAZON COM SERVICES LLC'
                , 'AMAZON.COM SERVICES INC':'AMAZON COM SERVICES LLC'
                , 'AMAZON FULFILLMENT SERVICES INC':'AMAZON COM SERVICES LLC'
                , 'AMAZON DEVELOPMENT CENTER US INC':'AMAZON COM SERVICES LLC'
                , 'AMAZON DATA SERVICES INC':'AMAZON COM SERVICES LLC'
                    , 'GOOGLE INC':'GOOGLE LLC'
           }

df23.WORKSITE_EMPLOYER = df23.WORKSITE_EMPLOYER.replace(dict_emp)  

In [17]:

# Fill in missing values with an empty string
df23['WORKSITE_EMPLOYER'] = df23['WORKSITE_EMPLOYER'].fillna('')

# Replace values in the 'WORKSITE_EMPLOYER' column based on condition contains('FACEBOOK')
mask = df23['WORKSITE_EMPLOYER'].str.contains('FACEBOOK')
df23.loc[mask, 'WORKSITE_EMPLOYER'] = 'META PLATFORMS INC'

In [18]:
df23

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,...,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_STATE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,WORKSITE_EMPLOYER
0,I-200-23090-892448,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,No,NaN,CA,113485.0,131000.0,Year,111322.0,Year,II,THERMO FISHER SCIENTIFIC INC.
1,I-200-23090-892621,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,Yes,"FedEx Corporate Services, Inc.",NY,109283.0,NaN,Year,109283.0,Year,II,"FEDEX CORPORATE SERVICES, INC."
2,I-200-23090-894024,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,No,NaN,NC,115000.0,125000.0,Year,101109.0,Year,II,"OPTCAPITAL, LLC"
3,I-200-23090-894211,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2141.00,Mechanical Engineers,Y,1,...,No,NaN,CA,75000.0,NaN,Year,70221.0,Year,I,"EXCELLENCE OPTO, INC."
4,I-200-23090-892630,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2112.00,Industrial Engineers,Y,1,...,No,NaN,MI,100000.0,NaN,Year,90667.0,Year,III,YANFENG INTERNATIONAL AUTOMOTIVE TECHNOLOGY US...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231539,I-200-22111-091232,Certified - Withdrawn,2022-04-20,2022-10-01,2022-04-27,H-1B,29-1069.03,Hospitalists,Y,1,...,No,NaN,OR,253050.0,NaN,Year,208000.0,Year,NaN,ASANTE PHYSICIAN PARTNERS
231540,I-200-22104-068433,Certified - Withdrawn,2022-04-13,2022-10-01,2022-04-20,H-1B,15-1132.00,"Software Developers, Applications",Y,1,...,No,NaN,TX,110000.0,180000.0,Year,109138.0,Year,III,"CLOUD TECHNOLOGIES, INC."
231541,I-200-21208-486017,Certified - Withdrawn,2021-07-27,2022-10-01,2021-08-03,H-1B,15-2041.01,Biostatisticians,Y,1,...,No,NaN,NH,75733.0,85733.0,Year,75733.0,Year,I,DARTMOUTH-HITCHCOCK CLINIC
231542,I-200-21043-075850,Certified - Withdrawn,2021-02-12,2022-10-01,2021-02-22,H-1B,13-2099.01,Financial Quantitative Analysts,Y,1,...,No,NaN,TX,125000.0,175000.0,Year,67371.0,Year,II,AI MX2 (US) LLC


In [19]:
df23['WORKSITE_EMPLOYER']=='META PLATFORMS INC'

0         False
1         False
2         False
3         False
4         False
          ...  
231539    False
231540    False
231541    False
231542    False
231543    False
Name: WORKSITE_EMPLOYER, Length: 221966, dtype: bool

In [20]:
#df23['WAGE_UNIT_OF_PAY'].unique()


In [21]:
#df23['PW_UNIT_OF_PAY'].unique()

In [22]:
# Add WAGE_RATE_OF_PAY_FROM_YR column

# df23['WAGE_RATE_OF_PAY_FROM_YR'] = df23['WAGE_RATE_OF_PAY_FROM']
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Hour', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 2000
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Month', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 12
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Week', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 50
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Bi-Weekly', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 25


# # Add PREVAILING_WAGE_YR column

# df23['PREVAILING_WAGE_YR'] = df23['PREVAILING_WAGE']
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Hour', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 2000
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Month', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 12
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Week', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 50
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Bi-Weekly', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 25

# # Add WAGE_DIFF column

# df23['WAGE_DIFF'] = (df23.WAGE_RATE_OF_PAY_FROM_YR- df23.PREVAILING_WAGE_YR)/df23.PREVAILING_WAGE_YR

# df23[['WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY'
#       , 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY'
#       , 'WAGE_RATE_OF_PAY_FROM_YR', 'PREVAILING_WAGE_YR', 'WAGE_DIFF']]


In [23]:
# df23['WAGE_DIFF'].max()

In [24]:
# df23['WAGE_DIFF'].min()

In [25]:
# len(df23[df23['WAGE_DIFF'] < 0])

In [26]:
rslt_df = df23[df23['WAGE_UNIT_OF_PAY'] == 'Year'] 
rslt_df

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,...,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_STATE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,WORKSITE_EMPLOYER
0,I-200-23090-892448,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,No,NaN,CA,113485.0,131000.0,Year,111322.0,Year,II,THERMO FISHER SCIENTIFIC INC.
1,I-200-23090-892621,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,Yes,"FedEx Corporate Services, Inc.",NY,109283.0,NaN,Year,109283.0,Year,II,"FEDEX CORPORATE SERVICES, INC."
2,I-200-23090-894024,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,No,NaN,NC,115000.0,125000.0,Year,101109.0,Year,II,"OPTCAPITAL, LLC"
3,I-200-23090-894211,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2141.00,Mechanical Engineers,Y,1,...,No,NaN,CA,75000.0,NaN,Year,70221.0,Year,I,"EXCELLENCE OPTO, INC."
4,I-200-23090-892630,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2112.00,Industrial Engineers,Y,1,...,No,NaN,MI,100000.0,NaN,Year,90667.0,Year,III,YANFENG INTERNATIONAL AUTOMOTIVE TECHNOLOGY US...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231539,I-200-22111-091232,Certified - Withdrawn,2022-04-20,2022-10-01,2022-04-27,H-1B,29-1069.03,Hospitalists,Y,1,...,No,NaN,OR,253050.0,NaN,Year,208000.0,Year,NaN,ASANTE PHYSICIAN PARTNERS
231540,I-200-22104-068433,Certified - Withdrawn,2022-04-13,2022-10-01,2022-04-20,H-1B,15-1132.00,"Software Developers, Applications",Y,1,...,No,NaN,TX,110000.0,180000.0,Year,109138.0,Year,III,"CLOUD TECHNOLOGIES, INC."
231541,I-200-21208-486017,Certified - Withdrawn,2021-07-27,2022-10-01,2021-08-03,H-1B,15-2041.01,Biostatisticians,Y,1,...,No,NaN,NH,75733.0,85733.0,Year,75733.0,Year,I,DARTMOUTH-HITCHCOCK CLINIC
231542,I-200-21043-075850,Certified - Withdrawn,2021-02-12,2022-10-01,2021-02-22,H-1B,13-2099.01,Financial Quantitative Analysts,Y,1,...,No,NaN,TX,125000.0,175000.0,Year,67371.0,Year,II,AI MX2 (US) LLC


In [27]:
# len(rslt_df[rslt_df['WAGE_DIFF'] < 0])

In [28]:
# rslt_df = rslt_df['WAGE_DIFF']
# rslt_df.hist(bins = 1000)

In [29]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221966 entries, 0 to 231543
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   CASE_NUMBER                     221966 non-null  object        
 1   CASE_STATUS                     221966 non-null  object        
 2   RECEIVED_DATE                   221966 non-null  datetime64[ns]
 3   DECISION_DATE                   221966 non-null  datetime64[ns]
 4   ORIGINAL_CERT_DATE              15520 non-null   datetime64[ns]
 5   VISA_CLASS                      221966 non-null  object        
 6   SOC_CODE                        221966 non-null  object        
 7   SOC_TITLE                       221966 non-null  object        
 8   FULL_TIME_POSITION              221966 non-null  object        
 9   TOTAL_WORKER_POSITIONS          221966 non-null  int64         
 10  EMPLOYER_NAME                   221966 non-null  object 

In [30]:
df23['PW_WAGE_LEVEL'].unique()

array(['II', 'I', 'III', 'IV', nan], dtype=object)

In [31]:
min(df23['DECISION_DATE'])

Timestamp('2022-10-01 00:00:00')

In [32]:
max(df23['DECISION_DATE'])

Timestamp('2023-03-31 00:00:00')

In [33]:
len(df23)-len(df23.drop_duplicates())

0

In [34]:
len(df23['CASE_NUMBER'])-len(df23['CASE_NUMBER'].drop_duplicates())

0

In [35]:
# Export df to compressed .csv

compression_opts = dict(method='zip', archive_name='h1b_lca_fy22.csv')  

# df23.to_csv('h1b_lca_fy22_data.zip', index=False, compression=compression_opts)  

In [36]:
df23

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,...,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_STATE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_LEVEL,WORKSITE_EMPLOYER
0,I-200-23090-892448,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,No,NaN,CA,113485.0,131000.0,Year,111322.0,Year,II,THERMO FISHER SCIENTIFIC INC.
1,I-200-23090-892621,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,Yes,"FedEx Corporate Services, Inc.",NY,109283.0,NaN,Year,109283.0,Year,II,"FEDEX CORPORATE SERVICES, INC."
2,I-200-23090-894024,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,15-1252.00,Software Developers,Y,1,...,No,NaN,NC,115000.0,125000.0,Year,101109.0,Year,II,"OPTCAPITAL, LLC"
3,I-200-23090-894211,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2141.00,Mechanical Engineers,Y,1,...,No,NaN,CA,75000.0,NaN,Year,70221.0,Year,I,"EXCELLENCE OPTO, INC."
4,I-200-23090-892630,Withdrawn,2023-03-31,2023-03-31,NaT,H-1B,17-2112.00,Industrial Engineers,Y,1,...,No,NaN,MI,100000.0,NaN,Year,90667.0,Year,III,YANFENG INTERNATIONAL AUTOMOTIVE TECHNOLOGY US...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231539,I-200-22111-091232,Certified - Withdrawn,2022-04-20,2022-10-01,2022-04-27,H-1B,29-1069.03,Hospitalists,Y,1,...,No,NaN,OR,253050.0,NaN,Year,208000.0,Year,NaN,ASANTE PHYSICIAN PARTNERS
231540,I-200-22104-068433,Certified - Withdrawn,2022-04-13,2022-10-01,2022-04-20,H-1B,15-1132.00,"Software Developers, Applications",Y,1,...,No,NaN,TX,110000.0,180000.0,Year,109138.0,Year,III,"CLOUD TECHNOLOGIES, INC."
231541,I-200-21208-486017,Certified - Withdrawn,2021-07-27,2022-10-01,2021-08-03,H-1B,15-2041.01,Biostatisticians,Y,1,...,No,NaN,NH,75733.0,85733.0,Year,75733.0,Year,I,DARTMOUTH-HITCHCOCK CLINIC
231542,I-200-21043-075850,Certified - Withdrawn,2021-02-12,2022-10-01,2021-02-22,H-1B,13-2099.01,Financial Quantitative Analysts,Y,1,...,No,NaN,TX,125000.0,175000.0,Year,67371.0,Year,II,AI MX2 (US) LLC
